# Approximating route from GPS positions

We have an unstructured dataset with 5k GPS positions of a bus.
Our goal is to compress the dataset to less than 100 representative points and extract the bus route. First, we define some functions. 

In [1]:
import folium
import pandas as pd, numpy as np, matplotlib.pyplot as plt
import random
from numpy import sin,cos,arctan2,sqrt,pi
from shapely.geometry import MultiPoint
from sklearn.cluster import DBSCAN
from geopy.distance import great_circle

####################################
## functions below are taken (and modified a bit) from blog post published by Geoff Boeing
## http://geoffboeing.com/2014/08/clustering-to-reduce-spatial-data-set-size/
####################################

def getDbScanClustersCenters(df, epsInKm, minObjects):
    clusters = getDbScanClusters(df, epsInKm, minObjects)
    center = getClustersCenters(clusters)
    
    #Reorder to Lat, Long
    for item in center:
        item[0], item[1] = item[1], item[0]
    return center
    
def getDbScanClusters(df, epsInKm, minObjects):
    coords = df.values
    kms_per_radian = 6371.0088
    epsilon = epsInKm / kms_per_radian
    db = DBSCAN(eps=epsilon, min_samples=minObjects, algorithm='ball_tree', metric='haversine').fit(np.radians(coords))
    cluster_labels = db.labels_
    print('Number of clusters: {}'.format(len(set(cluster_labels))))
    return pd.Series([coords[cluster_labels == n] for n in range(len(set(cluster_labels)) - 1)])

def get_centermost_point(cluster):
    centroid = (MultiPoint(cluster).centroid.x, MultiPoint(cluster).centroid.y)
    centermost_point = min(cluster, key=lambda point: great_circle(point, centroid).m)
    return centermost_point

def getClustersCenters(clusters):
    centermost_points = clusters.map(get_centermost_point)
    lats, lons = zip(*centermost_points)
    return pd.DataFrame({'lon':lons, 'lat':lats}).values.tolist()

####################################

def drawPointsOnMap(superMap, points, color, radiusSize):
    print('Drawing {} points on map'.format(len(points)))
    for point in points:
        folium.CircleMarker(point, radius = radiusSize, fill_color = color, color = color).add_to(superMap)
        
        
def getDistanceBetweenPoints(point1, point2):
        lon1 = point1[1] * pi / 180.0
        lon2 = point2[1] * pi / 180.0
        lat1 = point1[0] * pi / 180.0
        lat2 = point2[0] * pi / 180.0
        
        # haversine formula #### Same, but atan2 named arctan2 in numpy
        dlon = lon2 - lon1
        dlat = lat2 - lat1
        a = (sin(dlat/2))**2 + cos(lat1) * cos(lat2) * (sin(dlon/2.0))**2
        c = 2.0 * arctan2(sqrt(a), sqrt(1.0-a))
        km = 6371.0 * c
        return km
    
def getEndPoint(points):
    pointsForFindingLineEnd = list(points)
    currentPoint = random.choice(pointsForFindingLineEnd)

    pointsForFindingLineEnd.remove(currentPoint)

    while(len(pointsForFindingLineEnd) > 0):
        nextPoint = None
        distanceToNextPoint = float("inf")
        for point in pointsForFindingLineEnd:
            distance = getDistanceBetweenPoints(point, currentPoint)
            if (distance < distanceToNextPoint):
                nextPoint = point
                distanceToNextPoint= distance
    
        pointsForFindingLineEnd.remove(nextPoint)
        currentPoint = nextPoint
    
    return currentPoint

def getLineFromPoints(points, startingPoint):
    linePoints = list(points)
    currentPoint = startingPoint
    line = []
    while(len(linePoints) > 0):
        nextPoint = None
        distanceToNextPoint = float("inf")
        for point in linePoints:
            distance = getDistanceBetweenPoints(point,currentPoint)
            if (distance < distanceToNextPoint):
                nextPoint = point
                distanceToNextPoint= distance

        linePoints.remove(nextPoint)
        currentPoint = nextPoint
        line.append(currentPoint)

    return line


### As always we begin with a look at our data

In [ ]:
data = pd.read_csv('./data/locations.csv', index_col = False, header=0)
print('Number of positions: {}'.format(len(data)))
print('10 first entries:')
print(data[0:10])


### Okay, lets visualize it to get a better impression

In [ ]:
mapRaw = folium.Map(location=[51.127885, 17.05], zoom_start=13, tiles='OpenStreetMap')
drawPointsOnMap(mapRaw, data.values, '#3186cc', 10)
mapRaw

### The next step is to utilize DBSCAN from the scikit learn library

In [ ]:
clustersCenters = getDbScanClustersCenters(data, 0.05, 2)


mapC1 = folium.Map(location=[51.127885, 17.05], zoom_start=13, tiles='OpenStreetMap')
drawPointsOnMap(mapC1, data.values, '#3186cc', 5)
drawPointsOnMap(mapC1, clustersCenters, '#ff0000', 10)
mapC1

### We get clusters... but we have clusters on the detour (noise) and there are big gaps between the centers
### Maybe playing with the parameters might help...

In [ ]:
from ipywidgets import VBox, jsdlink, IntSlider
epsilon = IntSlider(description='Epsilon (in m)', min=10, max=300, value=20)
minPoints = IntSlider(description='Min-Points',min=2, max=50, value=2)
VBox([epsilon, minPoints])

In [ ]:
clustersCenters = getDbScanClustersCenters(data, epsilon.value /1000, minPoints.value)
mapC2 = folium.Map(location=[51.127885, 17.05], zoom_start=13, tiles='OpenStreetMap')
drawPointsOnMap(mapC2, clustersCenters, '#ff0000', 10)
mapC2

### The last step is to draw the rout between our centers

In [ ]:
endPoint = getEndPoint(clustersCenters)
mapRoute = folium.Map(location=[51.127885, 17.05], zoom_start=13, tiles='OpenStreetMap')
drawPointsOnMap(mapRoute, clustersCenters, '#ff0000', 10)

line = getLineFromPoints(clustersCenters, endPoint)

mapRoute.add_child(folium.PolyLine(locations = line, weight = 10, color="#156d08"))

mapRoute

### We reached our goal, the detour was removed and we got a small number of representatives. 